## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-26-21-30-40 +0000,nypost,Heartless killer seen pouring flammable liquid...,https://nypost.com/2025/07/26/us-news/pit-bull...
1,2025-07-26-21-30-04 +0000,bbc,Pubs and venues to be protected from noise com...,https://www.bbc.com/news/articles/cwye5jx8y3go
2,2025-07-26-21-26-16 +0000,startribune,Man found in Senate chamber at Minnesota State...,https://www.startribune.com/man-found-in-state...
3,2025-07-26-21-24-07 +0000,nyt,Saving for College Once Felt Essential. Some P...,https://www.nytimes.com/2025/07/26/business/52...
4,2025-07-26-21-13-05 +0000,nypost,NJ gov hopeful Mikie Sherrill dodges on Mamdan...,https://nypost.com/2025/07/26/us-news/mikie-sh...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2473/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
65,trump,34
92,new,15
120,gaza,12
125,aid,10
72,cambodia,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
63,2025-07-26-16-09-07 +0000,nypost,"Trump mediates Cambodia-Thailand conflict, dem...",https://nypost.com/2025/07/26/us-news/trump-me...,73
7,2025-07-26-21-07-33 +0000,nyt,Trump Says He Expects Quick Cease-Fire Between...,https://www.nytimes.com/2025/07/26/world/asia/...,67
114,2025-07-26-09-30-00 +0000,wsj,Now that President Trump’s One Big Beautiful B...,https://www.wsj.com/economy/trump-tax-cuts-win...,65
85,2025-07-26-12-48-43 +0000,cbc,Fighting on Thai-Cambodia border continues for...,https://www.cbc.ca/news/world/thailand-cambodi...,64
28,2025-07-26-19-30-55 +0000,wapo,Trump calls leaders of Thailand and Cambodia t...,https://www.washingtonpost.com/politics/2025/0...,63


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
63,73,2025-07-26-16-09-07 +0000,nypost,"Trump mediates Cambodia-Thailand conflict, dem...",https://nypost.com/2025/07/26/us-news/trump-me...
151,48,2025-07-26-01-00-00 +0000,wsj,The Democratic Party’s image has eroded to its...,https://www.wsj.com/politics/elections/democra...
180,39,2025-07-25-22-10-39 +0000,bbc,"'I witnessed war crimes' in Gaza, former worke...",https://www.bbc.com/news/videos/cy8k8045nx9o
103,32,2025-07-26-11-00-00 +0000,wsj,Faced with a rising number of threats and shak...,https://www.wsj.com/politics/policy/congress-s...
53,31,2025-07-26-16-54-10 +0000,nypost,Ukraine fends off major Russian advance on fro...,https://nypost.com/2025/07/26/world-news/ten-c...
114,31,2025-07-26-09-30-00 +0000,wsj,Now that President Trump’s One Big Beautiful B...,https://www.wsj.com/economy/trump-tax-cuts-win...
142,27,2025-07-26-02-23-22 +0000,nypost,Democratic Texas congressional candidate dragg...,https://nypost.com/2025/07/25/us-news/democrat...
168,26,2025-07-25-23-24-00 +0000,wsj,HHS Secretary RFK Jr. plans to remove all memb...,https://www.wsj.com/health/healthcare/rfk-heal...
164,24,2025-07-25-23-58-22 +0000,cbc,2 Southwest Airlines flight attendants hurt af...,https://www.cbc.ca/news/world/southwest-airlin...
5,22,2025-07-26-21-09-38 +0000,nypost,Debit cards given to NYC’s summer youth job pr...,https://nypost.com/2025/07/26/us-news/debit-ca...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
